### Generate grammar files from grammar trees

#### Idea

1. Given grammar trees(e.g. `devset.trees`) parse it to seperate each parenthesis surrounded strings.

    * For given `devset.trees`, the parenthesis are not sanitized.(e.g. `(( ()` which implies `( -> (`) This would result in failure of later processing. So we identified such cases in the grammar trees and sanitize '(' and ')' to '-LRB-' and '-RRB-' as in the result of stanford parser. 
    
2. Construct the tree by using `nltk.Tree.fromstring()`

3. Convert the tree to CNF by using `nltk.Tree.chomsky_normal_form()`

4. Traverse the tree by depth first search. Use a dictionary to keep counting the frequency of one symbol followed by another.

    * In the grammer trees, there are rules like `. -> .`, `, -> ,` where '.' and ',' are both nonterminal and ternimal. To address this problem, we sanitize nonterminal punctuations to things like 'PERIOD' 'COMMA'. And because of we previously sanitized '(' and ')' to '-LRB-' and '-RRB-', which results in rule like `-LRB- -> -LRB-`. So we sanitize the termial '-LRB-' and '-RRB-' to '(' and ')'.
    
    * In short, We make sure every non terminals are uppercase letters and there is no symbols that are both non terminal and terminal.
    
5. Output the grammar weighted by frequence to output grammar files `S1.gr` and `Vocab.gr`.